In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [52]:
baseUrl = 'http://www.10000recipe.com/recipe/'

In [69]:
def PageCrawler(recipeUrl):
    url = baseUrl + recipeUrl

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    recipe_title = []  # 레시피 제목
    recipe_source = {}  # 레시피 재료
    recipe_step = [] #레시피 순서

    try:
        res = soup.find('div', 'view2_summary')
        res = res.find('h3')
        recipe_title.append(res.get_text())
        res = soup.find('div', 'view2_summary_info')
        recipe_title.append(res.get_text().replace('\n', ''))
        res = soup.find('div', 'ready_ingre3')
    except(AttributeError):
        return

    # 재료 찾는 for문 가끔 형식에 맞지 않는 레시피들이 있어 try/ except 해준다
    try:
        for n in res.find_all('ul'):
            source = []
            title = n.find('b').get_text()
            recipe_source[title] = ''
            for tmp in n.find_all('li'):
                tempSource = tmp.get_text().replace('\n', '').replace(' ', ' ')
                source.append(tempSource.split("    ")[0])

            recipe_source[title] = source
    except (AttributeError):
        return
    
    # 요리순서 찾는 for문
    res = soup.find('div','view_step')
    i = 0
    for n in res.find_all('div','view_step_cont') :
        i = i + 1
        recipe_step.append('#' + str(i) + ' ' + n.get_text().replace('\n','').replace(' ',' '))   
        
    # 블로그 형식의 글은 스텝이 정확하게 되어있지 않기 때문에 제외해준다
    if not recipe_step :
        return

    recipe_all = [ recipe_title, recipe_source,recipe_step]  #제목, 재료, 순서
    return recipe_all

In [70]:
def CrawlingBetweenRanges(startRecipeId, endRecipeId):
    for i in range(startRecipeId, endRecipeId):
        if i % 10 == 0:
            print("count: " + str(i))
        res = PageCrawler(str(i))
        if res is None:
            continue
        return res
    
#        menuId = mydb.insert_menu(res[0][0], baseUrl+str(i))
#        for key, value in res[1].items():
#            for name in value:
#                if key == "[재료]" or key == "[양념]":
#                    mydb.insert_ingredient(menuId, name)

In [71]:
PageCrawler('6900135')

[['도시락 메뉴로 강추 불고기 또띠아 랩 (샌드위치) 만들기', '2인분 30분 이내 아무나 '],
 {'[재료]': ['또띠아', '불고기', '양상추', '로메인상추', '파프리카', '피망', '슬라이스치즈'],
  '[양념]': ['허니머스타드']},
 ['#1 재료를 준비해주세요~.',
  '#2 파프리카와 피망은 슬라이스해서 준비해주세요~.',
  '#3 슬라이스치즈는 반으로 잘라서 준비해주세요~.',
  '#4 양상추는 한입크기로 뜯어서 준비해주세요~.',
  '#5 불고기는 중불에서 다익을때까지 볶아주세요~.@6890083  (불고기 양념 레시피입니다)',
  '#6 마른팬에 중불로 앞뒤로 또띠아를 1분씩 구워주세요~.(생략가능하답니다~.)',
  '#7 또띠아에 하니머스타드를 한큰술 펴발라주세요~.',
  '#8 로메인상추를 2장 겹쳐서 올려주세요~.',
  '#9 양상추와 슬라이스 치즈도 올려주세요~.',
  '#10 불고기도 올려주세요~.',
  '#11 피망과 파프리카도 올려주세요~.',
  '#12 김밥을 말듯이 돌돌 말아주세요~.',
  '#13 호일 또는 기름종이로 감싸주면 먹기 더욱 간편하답니다~.',
  '#14 반으로 잘라주면 불고기 또띠아 랩 완성입니다~~~!!!♡♡♡']]